# **Question #1**

### Importing the needed libraries ti handle the data and do the calculations

In [1]:
import pandas as pd
from math import log
import operator
import numpy as np

### Building up a smale dataset with 14 rows in totall and 5 features with 1 ouput
### to start building the ID3 tree model

In [2]:
data = {'early_registeration':[1,1,0,0,0,0,1,0,0,1,1,0,0,1], 'finished_homework':[1,1,0,1,1,0,0,1,0,0,1,1,0,0], 'senior':[0,1,1,1,1,1,0,0,1,0,1,1,0,0], 'likes_coffe':[0,0,0,0,0,1,0,1,0,0,0,1,0,1], 'liked_the_last_homework':[1,1,0,1,0,1,1,1,1,0,0,1,1,0], 'a':[1,1,0,0,1,1,0,1,1,0,1,0,0,1]}
df = pd.DataFrame(data)
df

,early_registeration,finished_homework,senior,likes_coffe,liked_the_last_homework,a
0,1,1,0,0,1,1
1,1,1,1,0,1,1
2,0,0,1,0,0,0
3,0,1,1,0,1,0
4,0,1,1,0,0,1
5,0,0,1,1,1,1
6,1,0,0,0,1,0
7,0,1,0,1,1,1
8,0,0,1,0,1,1
9,1,0,0,0,0,0


### The ID3 decision tree mainly depends on calculating the information gain of each feature and by selecting the maximum gain from all the features we specify the feature corresponding to that gain as the node of the current position for further tree expansion 

### The information gain is measured by calculating the entropy of the class which is the indication of how much information does this feature ("Senior" for example) have to describe the output (A)

\begin{align}
Gain(S,F) = Entropy - \sum_{v \in Values(F)} \frac{\lvert{S_v}\rvert}{\lvert{S}\rvert} Entropy(S_v) \\
\\
Entropy(decision) = P_+ log_2 P_+ + P_- log_2 P_- \\
\end{align}

### First we have to calculate the output entropy first since it's gonna be used with all the features and won't change

In [3]:
def get_main_class_entropy(df):
    class_count = dict()
    class_prob = []
    class_entropy = 0.0
    for i in range(len(df.a.unique())):
        class_count[df.a.unique()[i]] = len(df[df.a == df.a.unique()[i]])
        class_prob.append(class_count[df.a.unique()[i]]/len(df))
    for i in range(len(class_prob)):
        class_entropy += -(class_prob[i])*log(class_prob[i],len(class_prob))
    #print(class_count)
    return class_entropy

In [4]:
class_entropy = get_main_class_entropy(df)
class_entropy

0.9852281360342516

### In order to get the entropy of each feature, first of all we have to describe each feature and count the contribution of each of it's classes with the output in order to calculate the probability and the entropy later on
### We represent the features and their counts as a dictionary the key is the feature's name and the values are as follows **The count of the first claas**, **The count of the second class**, **The count of the first class regarding the "0" output**, **The count of the first class regarding the "1" output**, **The count of the second class regarding the "0" output**, **The count of the second class regarding the "1" output**

In [5]:
features = dict()

# first major class count, second major class count, first first minor class count, first second minor class count,
# second first minor class count, second second minor class count
for i in range(df.shape[1]-1):
    first_class = df[df[df.iloc[:,i].name] == 0]
    first_class_first_class = first_class[first_class[first_class.iloc[:,-1].name] == 0]
    first_class_secound_class = first_class[first_class[first_class.iloc[:,-1].name] == 1]
    second_class = df[df[df.iloc[:,i].name] == 1]
    second_class_first_class = second_class[second_class[second_class.iloc[:,-1].name] == 0]
    second_class_second_class = second_class[second_class[second_class.iloc[:,-1].name] == 1]

    features[df.iloc[:,i].name] = [len(first_class), len(second_class), len(first_class_first_class), len(first_class_secound_class), len(second_class_first_class), len(second_class_second_class)]
print(features)

{'early_registeration': [8, 6, 4, 4, 2, 4], 'finished_homework': [7, 7, 4, 3, 2, 5], 'senior': [6, 8, 3, 3, 3, 5], 'likes_coffe': [10, 4, 5, 5, 1, 3], 'liked_the_last_homework': [5, 9, 2, 3, 4, 5]}


### Then we calculate the entropy of each class based on the dictionary made upove

In [6]:
def get_entropy(values):
    #print(values)
    if values[2] != 0:
        first_entropy = -(values[2]/values[0])*log(values[2]/values[0], 2)
    else:
        first_entropy = 0
    if values[3] != 0:
        first_entropy += -(values[3]/values[0])*log(values[3]/values[0], 2)
    else:
        first_entropy += 0
    if values[4] != 0:
        second_entropy = -(values[4]/values[1])*log(values[4]/values[1], 2)
    else:
        second_entropy = 0
    if values[5] != 0:
        second_entropy += -(values[5]/values[1])*log(values[5]/values[1], 2)
    else:
        second_entropy+= 0
    return first_entropy, second_entropy

\begin{align}
Entropy(decision) = P_+ log_2 P_+ + P_- log_2 P_- \\
\end{align}


In [7]:
entropy = dict()
for i in range(len(features)):
    first_ent, second_ent = get_entropy(features[list(features.keys())[i]])
    entropy[list(features.keys())[i]] = [first_ent, second_ent]
print(entropy)

{'early_registeration': [1.0, 0.9182958340544896], 'finished_homework': [0.9852281360342516, 0.863120568566631], 'senior': [1.0, 0.9544340029249649], 'likes_coffe': [1.0, 0.8112781244591328], 'liked_the_last_homework': [0.9709505944546686, 0.9910760598382222]}


### Next we calculate the gain of each feature

\begin{align}
Gain(S,F) = Entropy - \sum_{v \in Values(F)} \frac{\lvert{S_v}\rvert}{\lvert{S}\rvert} Entropy(S_v) \\
\end{align}

In [8]:
gain = dict()
for i in range(len(entropy)):
    feature_gain = class_entropy - (features[list(features.keys())[i]][0]/len(df))*entropy[list(entropy.keys())[i]][0] - (features[list(features.keys())[i]][1]/len(df))*entropy[list(entropy.keys())[i]][1]
    gain[list(features.keys())[i]] = feature_gain
print(gain)

{'early_registeration': 0.02024420715375619, 'finished_homework': 0.06105378373381032, 'senior': 0.011265848648557397, 'likes_coffe': 0.03914867190307081, 'liked_the_last_homework': 0.0013397424044413464}


### Getting the maximum gain we have

In [9]:
print('Maximum gain is:',gain[max(gain.keys(), key=(lambda k: gain[k]))])
print('The best feature is:', max(gain.keys(), key=(lambda k: gain[k])))

Maximum gain is: 0.06105378373381032
The best feature is: finished_homework


### To wrap it all in one function we get

In [10]:
def get_best_feature(df):

    class_entropy = get_main_class_entropy(df)
    #print(class_entropy)

    features = dict()

    # first major class count, second major class count, first first minor class count, first second minor class count,
    # second first minor class count, second second minor class count
    for i in range(df.shape[1]-1):
        first_class = df[df[df.iloc[:,i].name] == 0]
        first_class_first_class = first_class[first_class[first_class.iloc[:,-1].name] == 0]
        first_class_secound_class = first_class[first_class[first_class.iloc[:,-1].name] == 1]
        second_class = df[df[df.iloc[:,i].name] == 1]
        second_class_first_class = second_class[second_class[second_class.iloc[:,-1].name] == 0]
        second_class_second_class = second_class[second_class[second_class.iloc[:,-1].name] == 1]

        features[df.iloc[:,i].name] = [len(first_class), len(second_class), len(first_class_first_class), len(first_class_secound_class), 
        len(second_class_first_class), len(second_class_second_class)]

    entropy = dict()
    for i in range(len(features)):
        first_ent, second_ent = get_entropy(features[list(features.keys())[i]])
        entropy[list(features.keys())[i]] = [first_ent, second_ent]

    gain = dict()
    for i in range(len(entropy)):
        feature_gain = class_entropy - (features[list(features.keys())[i]][0]/len(df))*entropy[list(entropy.keys())[i]][0] - (features[list(features.keys())[i]][1]/len(df))*entropy[list(entropy.keys())[i]][1]
        gain[list(features.keys())[i]] = feature_gain
    #print(gain)
    print("Gain =" , gain[max(gain.keys(), key=(lambda k: gain[k]))])
    return max(gain.keys(), key=(lambda k: gain[k])), features[max(gain.keys(), key=(lambda k: gain[k]))]

### The following function represents a small implementation of the ID3 tree algorithm which represents the full statistics of the features and best feature and also we can pass the depth we want for the tree

### What's different in the following function is that we have a new reduced dataset at each node where the tree expands, this reduction is due to removing the corresponding feature within the classes

In [11]:
def decision_tree(df, levels):
    if levels == 1:
        feature, feature_info = get_best_feature(df)
        print("The best feature to split upon is: ", feature)
        print(feature, ': has',feature_info[0], 'in class 0' )
        print(feature, ': has',feature_info[1], 'in class 1' )
        print('class 0 has ', feature_info[2], 'of 0 classifications')
        print('class 0 has ', feature_info[3], 'of 1 classifications')
        print('class 1 has ', feature_info[4], 'of 0 classifications')
        print('class 1 has ', feature_info[5], 'of 1 classifications')

        classification_0 = feature_info[2:3].index(max([feature_info[2], feature_info[3]]))
        classification_1 = feature_info[4:].index(max([feature_info[4], feature_info[5]]))
        classification_0 += 2
        classification_1 += 4

        if classification_0 == 2:
            print("When it's 0, Prediction is 0")
        if classification_0 == 3:
            print("When it's 1, Prediction is 1")
        if classification_1 == 4:
            print("When it's 0, Prediction is 0")
        if classification_1 == 5:
            print("When it's 1, Prediction is 1")
        print("###############################")
    if levels == 2:
        print("This is the first LEVEL")
        print("..............................")
        feature, feature_info = get_best_feature(df)
        print(feature, ': has',feature_info[0], 'in class 0' )
        print(feature, ': has',feature_info[1], 'in class 1' )
        print('class 0 has ', feature_info[2], 'of 0 classifications')
        print('class 0 has ', feature_info[3], 'of 1 classifications')
        print('class 1 has ', feature_info[4], 'of 0 classifications')
        print('class 1 has ', feature_info[5], 'of 1 classifications')

        classification_0 = feature_info[2:3].index(max([feature_info[2], feature_info[3]]))
        classification_1 = feature_info[4:].index(max([feature_info[4], feature_info[5]]))
        classification_0 += 2
        classification_1 += 4

        if classification_0 == 2:
            print("When it's 0, Prediction is 0")
        if classification_0 == 3:
            print("When it's 1, Prediction is 1")
        if classification_1 == 4:
            print("When it's 0, Prediction is 0")
        if classification_1 == 5:
            print("When it's 1, Prediction is 1")

        # Dropping the features that was exracted and passing the new dataset to the next nodes    
        df1 = df[df[feature] == 0].drop(columns=[feature])
        df2 = df[df[feature] == 1].drop(columns=[feature])
        
        print("###############################")
        ###############################
        print("This is the second LEVEL")
        print("..............................")
        feature, feature_info = get_best_feature(df1)
        print(feature, ': has',feature_info[0], 'in class 0' )
        print(feature, ': has',feature_info[1], 'in class 1' )
        print('class 0 has ', feature_info[2], 'of 0 classifications')
        print('class 0 has ', feature_info[3], 'of 1 classifications')
        print('class 1 has ', feature_info[4], 'of 0 classification')
        print('class 1 has ', feature_info[5], 'of 1 classification')

        classification_0 = feature_info[2:3].index(max([feature_info[2], feature_info[3]]))
        classification_1 = feature_info[4:].index(max([feature_info[4], feature_info[5]]))
        classification_0 += 2
        classification_1 += 4

        if classification_0 == 2:
            print("When it's 0, Prediction is 0")
        if classification_0 == 3:
            print("When it's 1, Prediction is 1")
        if classification_1 == 4:
            print("When it's 0, Prediction is 0")
        if classification_1 == 5:
            print("When it's 1, Prediction is 1")
        ###############################
        print("###############################")
        feature, feature_info = get_best_feature(df2)
        print(feature, ': has',feature_info[0], 'in class 0' )
        print(feature, ': has',feature_info[1], 'in class 1' )
        print('class 0 has ', feature_info[2], 'of 0 classifications')
        print('class 0 has ', feature_info[3], 'of 1 classifications')
        print('class 1 has ', feature_info[4], 'of 0 classifications')
        print('class 1 has ', feature_info[5], 'of 1 classifications')
        
        

        classification_0 = feature_info[2:3].index(max([feature_info[2], feature_info[3]]))
        classification_1 = feature_info[4:].index(max([feature_info[4], feature_info[5]]))
        classification_0 += 2
        classification_1 += 4
        
        if classification_0 == 2:
            print("When it's 0, Prediction is 0")
        if classification_0 == 3:
            print("When it's 1, Prediction is 1")
        if classification_1 == 4:
            print("When it's 0, Prediction is 0")
        if classification_1 == 5:
            print("When it's 1, Prediction is 1")

### So now by simply passing the whole data to the upove wrapped up function and passing the depth level we get the below results

In [12]:
decision_tree(df, 1) # For depth of one, which means only the root

Gain =0.06105378373381032
The best feature to split upon is:  finished_homework
finished_homework : has 7 in class 0
finished_homework : has 7 in class 1
class 0 has  4 of 0 classifications
class 0 has  3 of 1 classifications
class 1 has  2 of 0 classifications
class 1 has  5 of 1 classifications
When it's 0, Prediction is 0
When it's 1, Prediction is 1
###############################


![title](depth1.png)

In [13]:
decision_tree(df, 2) # For depth of two, which means the root and the following branches

This is the first LEVEL
..............................
Gain = 0.06105378373381032
finished_homework : has 7 in class 0
finished_homework : has 7 in class 1
class 0 has  4 of 0 classifications
class 0 has  3 of 1 classifications
class 1 has  2 of 0 classifications
class 1 has  5 of 1 classifications
When it's 0, Prediction is 0
When it's 1, Prediction is 1
###############################
This is the second LEVEL
..............................
Gain = 0.46956521111470706
likes_coffe : has 5 in class 0
likes_coffe : has 2 in class 1
class 0 has  4 of 0 classifications
class 0 has  1 of 1 classifications
class 1 has  0 of 0 classification
class 1 has  2 of 1 classification
When it's 0, Prediction is 0
When it's 1, Prediction is 1
###############################
Gain = 0.2916919971380596
early_registeration : has 4 in class 0
early_registeration : has 3 in class 1
class 0 has  2 of 0 classifications
class 0 has  2 of 1 classifications
class 1 has  0 of 0 classifications
class 1 has  3 of 1 c

![title](depth2.png)